# Data Modeling with Cassandra

---

# Part I. ETL Pipeline for Pre-Processing Source Files

#### Import Python packages 

In [1]:
!pip install cassandra-driver

In [2]:
import pandas as pd
import re
import os
import glob
import numpy as np
import json
import csv

In [3]:
import cassandra

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)

# Create a for loop to create a list of files and collect each filepath
file_path_list = glob.glob(os.path.join(filepath,'*'))
print(file_path_list)

/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2
/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data
['/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data/2018-11-15-events.csv', '/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data/2018-11-22-events.csv', '/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data/2018-11-09-events.csv', '/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data/2018-11-18-events.csv', '/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data/2018-11-04-events.csv', '/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data/2018-11-01-events.csv', '/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data/2018-11-27-events.csv', '/Users/salomon.marquez/DATA ENGINEER PATH/NanoDegree Udacity/PROJECT 2/event_data/2018-11-10-eve

In [5]:
# Number of files to be processed
len(file_path_list)

30

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            ## print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [7]:
# check the number of rows in your csv file to be processed
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

---

# Part II. Create Apache Cassandra tables for querying data 

In this section you, 

- Create an Apache Cassandra cluster and keyspace
- Set keyspace
- Create tables based on specific queries
- Validate your data model querying data

#### Creating a Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
# TO-DO: Create a Keyspace 
try:
    session.execute(""" 
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [11]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Query 1

Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

```
SELECT artist_name, song_title, song_len FROM song_playlist WHERE session_id=338 AND item_in_session=4;
```

#### Create table
Table **song_playlist** uses a **COMPOSITE PRIMARY KEY**. Columns **session_id** and **item_in_session** integrate the **PARTITION KEY**. By using this KEY, session_id establishes a ONE to MANY relationship with item_in_session. As a result, it is more straightforward to look for a particular item_in_session element in a given session_id. 

In [12]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_playlist (
    session_id INT, 
    item_in_session INT, 
    artist_name TEXT, 
    song_title TEXT, 
    song_len FLOAT,
    PRIMARY KEY(session_id, item_in_session));""")

except Exception as e:
    print(e)

#### Insert data

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist (session_id, item_in_session, artist_name, song_title, song_len) "
        query = query + "VALUES(%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
try:
    rows = session.execute("""
    SELECT 
    artist_name,
    song_title,
    song_len
    FROM song_playlist
    WHERE session_id=338 AND item_in_session = 4;""")
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.song_len)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

```
SELECT artist_name, song_title, user_firstname, user_lastname FROM user_playlist WHERE user_id=10 AND sessionid=182 ORDER BY item_in_session ASC;
```

#### Create table
Table **user_playlist** uses a **COMPOSITE PRIMARY KEY**. Columns **session_id** and **item_in_session** integrate the **PARTITION KEY** and column **item_in_session** the **CLUSTERING KEY**. By using this COMPOSITE KEY, user_id establishes a ONE to MANY relationship with session_id enabling item_in_session to sorting data. This means that you can look for any playlist from a given user and sort the listened songs in that playlist.

In [15]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_playlist (
    user_id INT, 
    session_id INT, 
    item_in_session INT,
    artist_name TEXT, 
    song_title TEXT, 
    user_firstname TEXT,
    user_lastname TEXT,
    PRIMARY KEY((user_id, session_id), item_in_session));""")

except Exception as e:
    print(e)                    

#### Insert data

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_playlist (user_id, session_id, item_in_session, artist_name, song_title, user_firstname, user_lastname) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
try:
    rows = session.execute("""
    SELECT 
    artist_name,
    song_title, 
    user_firstname,
    user_lastname
    FROM user_playlist
    WHERE user_id=10 AND session_id=182
    ORDER BY item_in_session ASC;""")
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.user_firstname, row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3 
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

```
SELECT user_id, item_in_session, user_firstname, user_lastname, song_title FROM song_listeners WHERE song_title='All Hands Against His Own';
```

#### Create table
Table **song_listeners** uses a **COMPOSITE PRIMARY KEY**. Columns **song_title** and **user_id** integrate the **PARTITION KEY**. By using this KEY, song_title establishes a MANY to ONE relationship with user_id. As a result, it is more straightforward to look for a particular song_title element from user_id. 

In [18]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_listeners (
    song_title TEXT, 
    user_id INT,
    user_firstname TEXT,
    user_lastname TEXT,
    PRIMARY KEY(song_title, user_id));""")
except Exception as e:
    print(e)

#### Insert data

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (song_title, user_id, user_firstname, user_lastname) "
        query = query + "VALUES(%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [20]:
try:
    rows = session.execute("""
    SELECT 
    user_firstname,
    user_lastname
    FROM song_listeners
    WHERE song_title='All Hands Against His Own';""")
    
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user_firstname, row.user_lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Additional Query
Give me every user name (order by item_in_session DESC) in my music app history who listened to the song 'All Hands Against His Own'


#### Create table

In [21]:
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS song_listeners2 (
    song_title TEXT, 
    user_id INT,
    user_firstname TEXT,
    user_lastname TEXT,
    PRIMARY KEY(song_title, user_id))
    WITH CLUSTERING ORDER BY (user_id DESC);
    """)
except Exception as e:
    print(e)


#### Insert data

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners2 (song_title, user_id, user_firstname, user_lastname) "
        query = query + "VALUES(%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [23]:
try:
    rows = session.execute("""
    SELECT 
    user_firstname,
    user_lastname
    FROM song_listeners2
    WHERE song_title='All Hands Against His Own';""")
    
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user_firstname, row.user_lastname)

Sara Johnson
Tegan Levine
Jacqueline Lynch


### Drop the tables before closing out the sessions

In [24]:
try:
    session.execute("""
    DROP TABLE IF EXISTS song_playlist;
    """)
    
except Exception as e:
    print(e)

In [25]:
try:
    session.execute("""
    DROP TABLE IF EXISTS user_playlist;
    """)
    
except Exception as e:
    print(e)

In [26]:
try:
    session.execute("""
    DROP TABLE IF EXISTS song_listeners;
    """)
    
except Exception as e:
    print(e)

In [27]:
try:
    session.execute("""
    DROP TABLE IF EXISTS song_listeners2;
    """)
    
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()